In [1]:
!pip3 install torch==2.6.0
!pip3 install torch_geometric
!pip install wandb
# Optional dependencies:
!pip3 install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.6.0+cpu.html


[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cpu.html

[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [2]:
!pip install -r requirements.txt

  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiohttp-3.11.18-cp313-cp313-macosx_11_0_arm64.whl.metadata (7.7 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached appnope-0.1.4-py2.py3-none-any.whl.metadata (908 bytes)
  Using cached asttokens-3.0.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached charset_normalizer-3.4.2-cp313-cp313-macosx_10_13_universal2.whl.metadata (35 kB)
  Using cached comm-0.2.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached contourpy-1.3.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.0 MB/s eta 0:00:00
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run suc

In [3]:
import time
import logging
from util import create_parser, set_seed, logger_setup
from data_loading import get_data
from training import train_gnn
from inference import infer_gnn
import json
import argparse


In [4]:
from types import SimpleNamespace


args = SimpleNamespace(
    emlps=True, # if True, edge are updated during message passing
    reverse_mp=False,
    ports=False,
    tds=False,
    ego=False,
    batch_size=8192,
    n_epochs=100,
    num_neighs=[100, 100],
    seed=42,
    tqdm=False,
    data=".",
    model="pna", # Can be changed bewteen gin, gat, rgcn, pna
    testing=True,
    save_model=True, # Can be changed
    unique_name=False,
    finetune=False,
    inference=False
)


In [5]:
with open('data_config.json', 'r') as config_file:
    data_config = json.load(config_file)
# Setup logging
logger_setup()

#set seed
set_seed(args.seed)


2025-05-20 13:48:21,143 [INFO ] Random seed set as 42


## Get the training validation and testing data

In [6]:
from data_loading import get_data
t1 = time.perf_counter()

tr_data, val_data, te_data, tr_inds, val_inds, te_inds = get_data(args, data_config)

t2 = time.perf_counter()
logging.info(f"Retrieved data in {t2-t1:.2f}s")


2025-05-20 13:48:22,661 [INFO ] Available Edge Features: ['EdgeID', 'from_id', 'to_id', 'Timestamp', 'Amount Sent', 'Sent Currency', 'Amount Received', 'Received Currency', 'Payment Format', 'Is Laundering']
2025-05-20 13:48:40,204 [INFO ] Illicit ratio = 5177 / 5078345 = 0.10%
2025-05-20 13:48:40,206 [INFO ] Number of nodes (holdings doing transcations) = 515088
2025-05-20 13:48:40,206 [INFO ] Number of transactions = 5078345
2025-05-20 13:48:40,206 [INFO ] Edge features being used: ['Timestamp', 'Amount Received', 'Received Currency', 'Payment Format']
2025-05-20 13:48:40,206 [INFO ] Node features being used: ['Feature'] ("Feature" is a placeholder feature of all 1s)
2025-05-20 13:48:40,308 [INFO ] number of days and transactions in the data: 18 days, 5078345 transactions
2025-05-20 13:48:40,375 [INFO ] Calculate split: [[0, 1, 2, 3, 4, 5], [6, 7], [8, 9, 10, 11, 12, 13, 14, 15, 16, 17]]
2025-05-20 13:48:40,380 [INFO ] Total train samples: 63.98% || IR: 0.08% || Train days: [0, 1, 2,

## Training

In [9]:
if not args.inference:
    #Training
    logging.info(f"Running Training")
    train_gnn(tr_data, val_data, te_data, tr_inds, val_inds, te_inds, args, data_config)
else:
    #Inference
    logging.info(f"Running Inference")
    infer_gnn(tr_data, val_data, te_data, tr_inds, val_inds, te_inds, args, data_config)


2025-05-20 13:50:47,139 [INFO ] Running Training


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


ModuleNotFoundError: No module named 'tabulate'